# Mini Project 3 : Image Compression

How many times have we faced this issue? We love clicking images with our smartphone cameras and saving random photos off the web. And then one day – no space! Image compression helps deal with that headache.

In this mini-project, we investigage how linear algebra can be used to achieve efficient storage strategies, in particular in the context of image compression.

We aim here at using SVD, to minimize the size of an image in bytes to an acceptable level of quality. 
This means that you are able to store more images in the same disk space as compared to before.

Image compression takes advantage of the fact that only a few of the singular values obtained after SVD are large. You can trim the three matrices based on the first few singular values and obtain a compressed approximation of the original image. Some of the compressed images are nearly indistinguishable from the original by the human eye.





### Conversion from RGB to greyscale
    
    
 Although compression of colour images is similar to the compression of a greyscale image, we will use in this notebook
 black and white pictures only. To facilitate the import of colour pictures to your notebook, we provide hereafter a function  that converts pictures in Red/Green/Blue format to greyscale.   
      
    

In [21]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])


### Importing pictures to your Notebook

We will use here the "image" module, available in matplotlib : ``import matplotlib.image as mpimg ``


To import a picture, use the ``imread`` command:

img=mpimg.imread('filename.png') 

and make sure that the file is located in the same directory as you Jupyter Notebook.

This command stores in the variable ``img`` a colour map (picture). To convert to
a black and white image, use the function ``rgb2gray`` defined above:

`` bw = rgb2gray(img) ``  

Where ``bw`` is now a matrix, whose matrix elements represents a pixel with an intensity (0=black, 1=white).

To display within your Notebook the matrix, use the ``imshow`` command:

``plt.imshow(bw, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)``

The matrix ``bw`` can be manipulated with linear algebra operation as any other matrix. 


       
    
    
    

### Question 1 : SVD decomposition
 Load and display the figures "tiger.png" and "beach.png". After they are displayed in your Python Notebook,
    show the dimensions of the two matrices with the command :  ``numpy.matrix.shape(A)`` . For each matrix, apply the SVD decomposition, and plot in a graph the eigenvalues as function of their index (1,..,N). 
  










### Question 2 : Image compression, the "beach" picture
    
   
  Apply different cutoff in the SVD decomposition, by only keeping the largest eigenvalues in the SVD decomposition. Typically, if the decomposition reads:
  
  $$ A = U \Sigma V^T $$, 
  
  Where $A$ is the original matrix, we keep the L largest eigenvalues of $\Sigma$, and consider that the remaining ones are naught. 
  
  We suggest in the following steps to consider the following number of remaining components:

620, 500, 400, 300, 200, 100 .

For each component, recalculate the picture matrix, and display your result. 

    

### Question 3 : Image compression, the "tiger" picture


Extend what you have done in "Question 2" to the "tiger" picture.







## Question 4 : The mystery guest


We have today a mystery guest, identify who is hidden in the picture below. We have typically added noise
to a picture. For this, load the file "guest.png", following the same steps as in Question 2 and Question 3. 
However, this time the picture has been "damaged", and you'll have to use SVD to filter the noise. 

Use your own choice of cutoff on the $\Sigma$ matrix. Compare different number of components and try to identify our
"special guest". Choose the optimal number of components, such that the noise is reduced, but such that you can 
remain part of the information. 

Plot the SVD eigenvalues for the noisy picture. Discuss how you could identify the optimal number of components from this
graph, such that you retain information, but discard the noise. 





## Question 5 : The code


In this exercice, we try to decypher a written message. Load the image called "code.png",
and repeat the same step as in Question 4. Here, try to identify the hidden code in the picture.








## Open question: 

- Imagine that you are given a security video footage, where you see people coming in and out of the video frame. You would like to remove the still background, such that people moving in the frame can be more easily identified. Suggest how you could extend the SVD decomposition to achieve such a task (do not implement it, but discuss in general terms what would be your approach).
    

    
    
    
    
    


Answer: ......







